In [15]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [40]:
df = pd.read_csv("D:/Code/py_code/Artificial-Neural-Network/Single-Layer-Perceptron/data/train_data_cl.csv", header=None, names=list(range(13)))
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NaN
1,1,0,3,Braund,Mr. Owen Harris,male,22,1,0,A/5 21171,7.25,NaN,S
2,2,1,1,Cumings,Mrs. John Bradley (Florence Briggs Thayer),female,38,1,0,PC 17599,712.833,C85,C
3,3,1,3,Heikkinen,Miss. Laina,female,26,0,0,STON/O2. 3101282,7.925,NaN,S
4,4,1,1,Futrelle,Mrs. Jacques Heath (Lily May Peel),female,35,1,0,113803,53.1,C123,S


# Data Preprocessing

### Benerin column

In [ ]:
col_name = df.iloc[0,:12].values
col_name = np.insert(col_name, 4, "FirstName")

df.columns = col_name
df = df.drop(0).reset_index(drop=True)

df.head()

### Merge Name dan FirstName

In [ ]:
df["Name"] = df["FirstName"] + " " + df["Name"]
df = df.drop("FirstName", axis=1)
df

### Perbaikan data yang tidak sesuai

**Perbaikan Data Sex**

terdapat data dengan value yang bukan 'male' atau 'female'

In [ ]:
print(df['Sex'].unique().tolist())
true_sex_val = ['male', 'female']
count = 0
for val in df["Sex"].values:
    if val not in true_sex_val:
        df.loc[count, 'Sex'] = np.random.choice(true_sex_val)
    count += 1
print(df['Sex'].unique().tolist())

**Perbaikan Data Fare**

terdapat data dengan value yang bukan float

In [ ]:
count = 0
for val in df['Fare'].values:
    try:
        float(val)
    except ValueError:
        df.loc[count, 'Fare'] = 0
    count += 1

**Perbaikan Data Parch**

terdapat data dengan value yang bukan int

In [ ]:
count = 0
for val in df['Parch'].values:
    if len(val) > 1:
        df.loc[count, 'Parch'] = 0
    count += 1

### Change Data Type

In [ ]:
df.info()

In [ ]:
df = df.astype({'PassengerId': 'int64', 'Survived': 'int64', 'Pclass': 'int64', 'SibSp': 'int64', 'Parch': 'int64'})
df = df.astype({'Name': 'string', 'Sex': 'string', 'Ticket': 'string', 'Cabin': 'string', 'Embarked': 'string'})
df = df.astype({'Age': 'float64', 'Fare': 'float64'})
df.info()

### Menghilangkan Column yang tidak dipakai

In [ ]:
del df['Name']
del df['Ticket']
del df['Cabin']

### Cek & Replace Missing Value

In [ ]:
missing = pd.DataFrame({
    'total' : df.isnull().sum(),
    'percent' : df.isnull().sum()/df.shape[0] * 100})
    
missing

In [ ]:
df['Embarked'] = df['Embarked'].fillna(value=np.random.choice(df['Embarked'].unique().tolist()))
df['Fare'] = df['Fare'].fillna(value=0)
df.isna().sum()
clean_data = df.copy()

### Encode

In [ ]:
lbenc = LabelEncoder()

for i in df.columns.values:
    if df[i].dtypes == 'string':
        df[i] = lbenc.fit_transform(df[i])

df.head()

### Split Feature and Label

In [ ]:
train = df.copy()
x_train = train.iloc[:, 2:].values
y_train = df.iloc[:, 1].values
train

### Normalisasi

In [ ]:
feature = x_train.copy()
label = y_train.copy()
label = label.reshape(-1,1)
passengerId = train.iloc[:, 0].values.reshape(-1,1)
col = train.columns.values.tolist()
col.pop(1)
col.append("Survived")

mnmx = MinMaxScaler()
feature = mnmx.fit_transform(feature)

data = np.concatenate((passengerId, feature), axis=1)
dataLabel = np.concatenate((data, label), axis=1)
normalize_data = pd.DataFrame(dataLabel, columns=col)

normalize_data = normalize_data.astype({'PassengerId': 'int64', 'Survived': 'int64'})
normalize_data

### Export hasil normalisasi ke excel

In [ ]:
with pd.ExcelWriter('Single Layer Perceptron.xlsx') as writer:
    clean_data.to_excel(writer, sheet_name='Hasil Preprocessing')
    df.to_excel(writer, sheet_name='Hasil Encode')
    normalize_data.to_excel(writer, sheet_name='Normalisasi')